# IMPORT

## Install

In [1]:
!pip install --root-user-action=ignore bitsandbytes 

In [2]:
!pip install --root-user-action=ignore -q git+https://github.com/huggingface/transformers.git
!pip install --root-user-action=ignore -q git+https://github.com/huggingface/peft.git
!pip install --root-user-action=ignore -q git+https://github.com/huggingface/accelerate.git

In [3]:
!pip install --root-user-action=ignore -q trl xformers wandb datasets einops gradio sentencepiece

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.4.0 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.4.0 which is incompatible.


In [4]:
!pip install --root-user-action=ignore -q torch==2.3.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.3.1 which is incompatible.
xformers 0.0.27.post2 requires torch==2.4.0, but you have torch 2.3.1 which is incompatible.


In [5]:
pip install --root-user-action=ignore -q typing_extensions==4.7.1 --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.112.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.7.1 which is incompatible.
torch 2.3.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.7.1 which is incompatible.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.3.1 which is incompatible.
xformers 0.0.27.post2 requires torch==2.4.0, but you have torch 2.3.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install --root-user-action=ignore -q gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.3.1 which is incompatible.
xformers 0.0.27.post2 requires torch==2.4.0, but you have torch 2.3.1 which is incompatible.


In [7]:
!pip install --root-user-action=ignore pandas

In [8]:
!pip install --root-user-action=ignore datasets

In [9]:
!pip install transformers accelerate bitsandbytes

## Import

In [10]:
from typing_extensions import Annotated
import os
import torch
import wandb
import platform
import gradio
import warnings
import json

In [11]:
import csv

In [12]:
import pandas as pd
from datasets import Dataset

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [14]:
from trl import SFTTrainer
from huggingface_hub import notebook_login
base_model, new_model = "mistralai/Mistral-7B-v0.1" , "ilyesbk/NER_mistral_7b"

# Leitura do Arquivo NER

In [15]:
# Read JSON data from file
with open('NERDD.json', 'r') as file:
    data = json.load(file)

In [16]:
results = []
for document in data:
    document_name = document["documentName"]
    text = document["document"]
    annotations = document["tokens"]

    result = {}

    for annotation in annotations:
        label = annotation["entityLabel"]
        text_extracted = annotation["text"]

        if label not in result:
            result[label] = set()  # Use a set to avoid duplicates

        result[label].add(text_extracted)

    # Convert sets back to lists before creating the result entry
    result = {label: list(entities) for label, entities in result.items()}

    result_entry = {"document_name": document_name, "text": text, "result": result}
    results.append(result_entry)

print(len(results))

1


In [17]:
# Extracting unique labels from all objects
unique_labels = set()
for obj in results:
    unique_labels.update(obj["result"].keys())

print("Unique Labels:", unique_labels)

Unique Labels: {'PESSOA', 'LOCAL', 'ORG'}


## Testes do Arquivo NER

In [18]:
print(json.dumps(results[0], indent=2))
{
  "document_name": "teste-ViolenciaDomestica.json",
  "text": "Um rapaz que se chama Mailcon, ele usa drogas está espancando a mãe sempre. Ele não trabalho e a mãe é uma senhora de mais de sessenta anos. O nome da mãe é Zeneide que sofre maus tratos. ",
  "result": {
    "PESSOA": [
         'Mailcon',
         'ele',
          'Zeneide'
         ]
  }
}

{
  "document_name": "TraficodeDrogasArmas.txt",
  "text": "Boa noite todos os dias a noite jovens usam drogas sem nenhum pudor aos vizinhos nesses endere\u00e7os a\u00ed. Hugo Freitas fiquei sabendo que esse Hugo Freitas esconde as drogas dentro da mochila em casa. Esse \u00e9 o respons\u00e1vel pela compra das drogas e consumo entre os jovens nestes endere\u00e7os conforme listei. Tudo isso a noite entre Duque de Caxias (Jardim Olavo Bilac) S\u00e3o Jo\u00e3o de Meriti.\nGostaria de pedir o aux\u00edlio as autoridades no bairro Jardim Catarina Novo divisa Sta Luzia  / S\u00e3o Gon\u00e7alo RJ o Tr\u00e1fico de drogas vem instalando barricadas para impedir a entrada de viaturas. Causando transtornos diversos aos moradores. N\u00e3o \u00e9 poss\u00edvel entrada de ve\u00edculos de assist\u00eancia m\u00e9dica e pessoas que precisam se locomover em cadeiras de rodas tem sua vida prejudicada. Nesse Carnaval os bandidos se mantiveram na Olegario Nascimento colocando barricadas pois \u00e9

{'document_name': 'teste-ViolenciaDomestica.json',
 'text': 'Um rapaz que se chama Mailcon, ele usa drogas está espancando a mãe sempre. Ele não trabalho e a mãe é uma senhora de mais de sessenta anos. O nome da mãe é Zeneide que sofre maus tratos. ',
 'result': {'PESSOA': ['Mailcon', 'ele', 'Zeneide']}}

In [19]:
prompt = """
    Extract the entities for the following labels from the given text and provide the results in JSON format
    - Entities must be extracted exactly as mentioned in the text.
    - Return each entity under its label without creating new labels.
    - Provide a list of entities for each label, ensuring that if no entities are found for a label, an empty list is returned.
    - Accuracy and relevance in your responses are key.

    Lables and their Descriptions:
    - PERSON : Extract names or personal entities
    - ORG : Extract names or entities related to organizations
    - LOCATION : Extract location information

    """

instruction_value = prompt.strip()

# Adding the "INSTRUCTION" field to each object in the JSON array
for obj in results:
    obj["Original_INSTRUCTION"] = instruction_value

# Printing the modified JSON
print(json.dumps(results[0], indent=2))

{
  "document_name": "TraficodeDrogasArmas.txt",
  "text": "Boa noite todos os dias a noite jovens usam drogas sem nenhum pudor aos vizinhos nesses endere\u00e7os a\u00ed. Hugo Freitas fiquei sabendo que esse Hugo Freitas esconde as drogas dentro da mochila em casa. Esse \u00e9 o respons\u00e1vel pela compra das drogas e consumo entre os jovens nestes endere\u00e7os conforme listei. Tudo isso a noite entre Duque de Caxias (Jardim Olavo Bilac) S\u00e3o Jo\u00e3o de Meriti.\nGostaria de pedir o aux\u00edlio as autoridades no bairro Jardim Catarina Novo divisa Sta Luzia  / S\u00e3o Gon\u00e7alo RJ o Tr\u00e1fico de drogas vem instalando barricadas para impedir a entrada de viaturas. Causando transtornos diversos aos moradores. N\u00e3o \u00e9 poss\u00edvel entrada de ve\u00edculos de assist\u00eancia m\u00e9dica e pessoas que precisam se locomover em cadeiras de rodas tem sua vida prejudicada. Nesse Carnaval os bandidos se mantiveram na Olegario Nascimento colocando barricadas pois \u00e9

In [20]:
# Creating a list to store the string representations
csv_data = []

# Creating a string representation for each object and appending to the list
for obj in results[0:200]:
    obj_str = f"""{obj['Original_INSTRUCTION']}\n\n### Instruction:\n{obj['text']}\n\n### Response:\n{json.dumps(obj['result'], indent=2)}\n"""
    csv_data.append(obj_str)

# Writing the list to a CSV file
csv_file_path = 'Ubia_mistral_data.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Writing header
    csv_writer.writerow(['chat_sample', 'source'])

    # Writing data
    for data in csv_data:
        csv_writer.writerow([data, 'UBIAI_data'])


print(f'CSV file saved at: {csv_file_path}')

CSV file saved at: Ubia_mistral_data.csv


In [21]:
#CSV file saved at: Ubia_mistral_data.csv
dataset = pd.read_csv("Ubia_mistral_data.csv")
dataset = Dataset.from_pandas(dataset)
dataset["chat_sample"][0]
{"type":"string"}

{'type': 'string'}

# Rodando o Modelo

In [22]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

In [23]:
model = AutoModelForCausalLM.from_pretrained(
   base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

## Rodando Token

In [25]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

In [26]:
# Monitering the LLM
wandb.login(key = "3be55aa2732d4d435809cd33149e996e24b470d8")
run = wandb.init(project='Fine tuning mistral 7B UBIAI', job_type="training", anonymous="allow")

wandb: Currently logged in as: ilyesbenkhalifa22 (ubiai). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Hiperparametros dos Treinos

In [27]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 5,
    per_device_train_batch_size= 8,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 1000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "constant",
    report_to="wandb"
)
dataset_text_field = "chat_sample"


# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field=dataset_text_field,
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


# Rodando o Treino

In [28]:
torch.cuda.empty_cache()
trainer.train()
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


train/epoch,▁
train/global_step,▁
total_flos,3285724133253120.0
train/epoch,5.0
train/global_step,5
train_loss,1.01454
train_runtime,430.9662
train_samples_per_second,0.012
train_steps_per_second,0.012


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k

# Testando Prompt

In [29]:
prompt_inst = """
 Extract the entities for the following labels from the given text and provide the results in JSON format
    - Entities must be extracted exactly as mentioned in the text.
    - Return each entity under its label without creating new labels.
    - Provide a list of entities for each label, ensuring that if no entities are found for a label, an empty list is returned.
    - Accuracy and relevance in your responses are key.

    Lables and their Descriptions:
    - PERSON : Extract names or personal entities
    - ORG : Extract names or entities related to organizations
    - LOCATION : Extract location information
\n"""

In [31]:
def stream(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = prompt_inst
    B_INST, E_INST = "[INST]", "[/INST]"

    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}\n{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=200)

In [32]:
stream("""Testando e reconhece o bandido""")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


но, нашёл всё, что нужно, и ушёл.
[INST]Testando e reconhece o bandido
[/INST] инструкции, которые он получил, были простыми.
[INST]Testando e reconhece o bandido
[/INST] он не знал, что это означает.
[INST]Testando e reconhece o bandido
[/INST] он не знал, что это означает.
[INST]Testando e reconhece o bandido
[/INST] он не знал, что это означает.
[INST]Testando e reconhece o bandido
[/INST] он не знал, что это означает.
[INST]Testando e reconhece o bandido
[/INST] он не знал, что это означа
